Tutorial: https://www.youtube.com/playlist?list=PL58zEckBH8fCMIVzQCRSZVPUp3ZAVagWi

In [3]:
# just running the game
import gymnasium as gym
import flappy_bird_gymnasium

env = gym.make('FlappyBird-v0', render_mode='human', use_lidar=False)

obs, _ = env.reset()

while True:
    action = env.action_space.sample()
    obs, reward, done, _, info = env.step(action)
    if done:
        break
    
env.close()

Video 2 notes
A Deep Q Network is a regular deep NN - sounds like the replay stuff is outside the neural network

Inputs for flappy birds are the position information for the pipes and bird,  the outputs are the Q values for flap or not - the expected reward for each action


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DQN(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=256):
        super(DQN, self).__init__()
        # defining the layers
        # fc1 is the transformation from state (input) to hidden layer
        # fc2 is the transformation from hidden layer to action (output)
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, action_dim)
        
    def forward(self, x):
        # forward pass through the network
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# run the NN one time through
state_dim = 12 # 12 inputs
action_dim = 2 # 2 outputs
net = DQN(state_dim, action_dim) # create the network
 # generate random states, first dimension is batch size (number of rand states)
state = torch.randn(10, state_dim)
output = net(state) # forward pass through the network
print(state)

tensor([[ 1.6189e-01,  1.4241e-01,  1.4276e+00,  7.4891e-02,  2.6081e+00,
          1.0510e+00,  7.7944e-01, -9.0722e-01, -4.3193e-01,  2.0921e-01,
          1.0838e+00, -3.4845e-01],
        [ 1.6901e-01, -1.6374e+00, -5.6085e-02,  7.3569e-01, -1.0515e-01,
          1.5609e+00,  8.2857e-01,  1.4544e+00,  2.7310e-01,  1.2579e+00,
         -1.4739e+00,  2.0534e+00],
        [-6.7231e-01, -3.8325e-02, -1.0731e+00,  1.6503e+00,  1.0511e+00,
          1.3089e+00,  2.6337e-01,  5.1138e-01,  9.6702e-01,  1.1805e+00,
         -2.5697e-01,  1.5073e+00],
        [ 1.2611e+00, -1.7244e+00,  6.2457e-01,  9.6155e-01,  4.3024e-01,
         -1.1364e+00, -1.1045e+00, -5.8531e-01,  4.6322e-01,  1.0190e+00,
         -1.3336e-02,  3.9228e-01],
        [-2.8503e-01,  4.6871e-01,  9.7948e-01, -1.1783e+00, -7.4249e-01,
         -8.7258e-01,  4.6423e-01, -1.6245e+00, -1.4352e-01, -2.7235e-01,
          7.4771e-01,  8.0418e-01],
        [-2.2352e+00, -4.2880e-01, -1.0118e+00,  2.7551e-02, -3.0074e-01,
      

In [6]:
# make a class for the agent

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Agent:
    def run (self, is_train, render=False):
        # run the agent
        env = gym.make('FlappyBird-v0', render_mode='human' if render else None, use_lidar=False)
        
        state_dim = env.observation_space.shape[0]
        action_dim = env.action_space.n
        
        policy_net = DQN(state_dim, action_dim).to_device(device)
        
        obs, _ = env.reset()
        while True:
            action = env.action_space.sample()
            obs, reward, done, _, info = env.step(action)
            if done:
                break
        env.close()

Video 3 notes

Experience replay:
  - an experience is defined as a tuple of (state, action, reward, next_state, terminated)
  - save these experiences in a replay buffer (first in first out)
  - the epsilon ( $\epsilon$ ) greedy policy is used to select the action
    - $\epsilon$ is the probability of selecting a random action, else the best action is selected
    - kinda of a stochastic annealing type thing
  

In [7]:
# replay memory
from collections import deque
import random

class ReplayMemory:
    def __init__(self, capacity, seed=None):
        self.memory = deque(maxlen=capacity)
        if seed is not None:
            random.seed(seed)
        
    def append(self, transition): # transition is a tuple of (state, action, next_state, reward, done)
        # append a transition to the buffer
        self.memory.append(transition)
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [11]:
import itertools
import yaml

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Agent:
    def __init__(self, hyperparam_option):
        with open('hyperparameters.yml', 'r') as f:
            all_hyperparams = yaml.safe_load(f)
            self.hyperparams = all_hyperparams[hyperparam_option]
            self.replay_memory_size = self.hyperparams['replay_memory_size'] # size of the replay memory
            self.mini_batch_size = self.hyperparams['mini_batch_size'] # size of the training data set sampled from the replay memory
            self.epsilon_init = self.hyperparams['epsilon_init'] # proportion of actions that are random
            self.epsilon_decay = self.hyperparams['epsilon_decay'] # decay rate of epsilon
            self.epsilon_min = self.hyperparams['epsilon_min'] # minimum value of epsilon
    
    
    
    def run (self, is_train, render=False):
        # run the agent
        env = gym.make('FlappyBird-v0', render_mode='human' if render else None, use_lidar=False)
        
        state_dim = env.observation_space.shape[0]
        action_dim = env.action_space.n
        
        rewards_per_episode = []
        epsilon_history = []
        
        policy_net = DQN(state_dim, action_dim).to(device)
        
        if is_train:
            memory = ReplayMemory(capacity=10000)
            epsilon = self.epsilon_init
        
        
        
        for episode in itertools.count():
            state, _ = env.reset()
            # convert anything going into the network to a tensor
            state = torch.tensor(state, dtype=torch.float, device=device).to(device)
            
            episode_reward = 0.0
            done = False
            while not done:
                # Picking an action
                if is_train and random.random() < epsilon:
                    action = env.action_space.sample()
                    action = torch.tensor(action, dtype=torch.int64, device=device)
                else:
                    with torch.no_grad():
                        action = policy_net(state.unsqueeze(dim=0)).squeeze().argmax()
                
                # Processing
                new_state, reward, done, _, info = env.step(action.item())
                
                # accumulate reward
                episode_reward += reward
                
                # convert new state and reward to tensors on device
                new_state = torch.tensor(new_state, dtype=torch.float, device=device)
                reward = torch.tensor(reward, dtype=torch.float, device=device)
                
                if is_train:
                    memory.append((state, action, new_state, reward, done))
                    
                state = new_state
                
            rewards_per_episode.append(episode_reward)
            
            # in this implementation we're using a geometric decay for epsilon (taking the product of epsilon_decay and current epsilon)
            # a linear decay is another option, decreasing epsilon by a fixed amount each episode (adjust epsilon_decay hyperparameter accordingly)
            epsilon = max(epsilon * self.epsilon_decay, self.epsilon_min)
            epsilon_history.append(epsilon)
        

In [13]:
agent = Agent('cartpole1')
agent.run(is_train=True)

Video 5 notes
- Now we have a policy network, which determines the action we should take
- We need a target network, which is used to calculate future Q values
- Every once in a while we copy the policy network over to the training network
- This stabilizes the training process, so that the network used to estimate outcomes doesn't change every time we try to make a decision

In [ ]:
import itertools
import yaml

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Agent:
    def __init__(self, hyperparam_option):
        with open('hyperparameters.yml', 'r') as f:
            all_hyperparams = yaml.safe_load(f)
            self.hyperparams = all_hyperparams[hyperparam_option]
            self.replay_memory_size = self.hyperparams['replay_memory_size'] # size of the replay memory
            self.mini_batch_size = self.hyperparams['mini_batch_size'] # size of the training data set sampled from the replay memory
            self.epsilon_init = self.hyperparams['epsilon_init'] # proportion of actions that are random
            self.epsilon_decay = self.hyperparams['epsilon_decay'] # decay rate of epsilon
            self.epsilon_min = self.hyperparams['epsilon_min'] # minimum value of epsilon
            self.network_sync_rate = self.hyperparams['network_sync_rate'] # how often to update the target network
            self.learning_rate_a = self.hyperparams['learning_rate_a'] 
            self.discount_factor_g = self.hyperparams['discount_factor_g'] # how much to discount future rewardsvs sooner rewards
            self.loss_fn = nn.MSELoss() # loss function (mean squared error)
            self.optimizer = None
    
    
    
    def run (self, is_train, render=False):
        # run the agent
        env = gym.make('FlappyBird-v0', render_mode='human' if render else None, use_lidar=False)
        
        state_dim = env.observation_space.shape[0]
        action_dim = env.action_space.n
        
        rewards_per_episode = []
        epsilon_history = []
        
        policy_net = DQN(state_dim, action_dim).to(device)
        
        if is_train:
            memory = ReplayMemory(capacity=10000)
            epsilon = self.epsilon_init
            target_net = DQN(state_dim, action_dim).to(device)
            target_net.load_state_dict(policy_net.state_dict())
            
            step_count = 0
            
            # policy network optimizer
            self.optimizer = torch.optim.Adam(policy_net.parameters(), lr=self.learning_rate_a)
        
        
        
        for episode in itertools.count():
            state, _ = env.reset()
            # convert anything going into the network to a tensor
            state = torch.tensor(state, dtype=torch.float, device=device).to(device)
            
            episode_reward = 0.0
            done = False
            while not done:
                # Picking an action
                if is_train and random.random() < epsilon:
                    action = env.action_space.sample()
                    action = torch.tensor(action, dtype=torch.int64, device=device)
                else:
                    with torch.no_grad():
                        action = policy_net(state.unsqueeze(dim=0)).squeeze().argmax()
                
                # Processing
                new_state, reward, done, _, info = env.step(action.item())
                
                # accumulate reward
                episode_reward += reward
                
                # convert new state and reward to tensors on device
                new_state = torch.tensor(new_state, dtype=torch.float, device=device)
                reward = torch.tensor(reward, dtype=torch.float, device=device)
                
                if is_train:
                    memory.append((state, action, new_state, reward, done))
                    
                    step_count += 1
                    
                state = new_state
                
            rewards_per_episode.append(episode_reward)
            
            # in this implementation we're using a geometric decay for epsilon (taking the product of epsilon_decay and current epsilon)
            # a linear decay is another option, decreasing epsilon by a fixed amount each episode (adjust epsilon_decay hyperparameter accordingly)
            epsilon = max(epsilon * self.epsilon_decay, self.epsilon_min)
            epsilon_history.append(epsilon)
            
            if len(memory) >= self.mini_batch_size:
                mini_batch = memory.sample(self.mini_batch_size)
                
                self.optimize(mini_batch, policy_net, target_net)
                
                if step_count > self.network_sync_rate:
                    target_net.load_state_dict(policy_net.state_dict())
                    step_count = 0
    
    def optimize(self, mini_batch, policy_net, target_net):
        for state, action, new_state, reward, done in mini_batch:
            if done:
                target_q = reward
            else:
            # calculate the target value
                with torch.no_grad():
                    target_q = reward + self.discount_factor_g * target_net(new_state).max()
            
            current_q = policy_net(state)
            
            loss = self.loss_fn(current_q, target_q)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

left off at 5:30 here: https://www.youtube.com/watch?v=vYRpJo-KMSw - next step is to define the optimize function for Agent